# Nations and Conflict

In [1]:
import pandas as pd
import numpy as np
import pickle

import altair as alt


# importing local libraries for drawings (because they clutter)
# in Github, you can search  
#   They should still be available within the git repo but it's pretty much template stuff from altair
import sys
sys.path.append('../')
from lib.builders import towns,nations
import lib.drawMaps as draw

with (open('world_2.p', 'rb')) as pickle_file:
    world = pickle.load(pickle_file)
    
world

Towns are python objects, each with thier own diplomacy. Land spaces are attributed to nations in a previous stage. 

In [2]:
world.towns

[town of Stenchjaw: population: 4 location: [20,22] founded 0,
 capitol of Flamemoral: population: 2 location: [7,6] founded 1,
 capitol of Waterblaze: population: 4 location: [38,39] founded 1,
 capitol of Magegarden: population: 6 location: [22,22] founded 2,
 capitol of Ragecombe: population: 5 location: [23,6] founded 2,
 town of Oakmore: population: 2 location: [23,1] founded 3,
 capitol of Stoneglen: population: 2 location: [18,16] founded 3,
 town of Lionmain: population: 3 location: [6,11] founded 3,
 town of King'sspring: population: 2 location: [38,19] founded 4,
 capitol of Heirhollow: population: 4 location: [29,23] founded 5,
 capitol of Warplace: population: 4 location: [10,14] founded 5,
 town of Fenvale: population: 1 location: [27,7] founded 6,
 town of Waterhollow: population: 2 location: [34,36] founded 7,
 town of Lionhollow: population: 2 location: [6,15] founded 7,
 town of Fallrail: population: 1 location: [27,6] founded 7,
 capitol of Brinehail: population: 3 lo

In [3]:
town = world.towns[1]
print(f"{town.name} is loyal to the nation of {town.nation}")
print(f"{town.diplomacy}")

Flamemoral is loyal to the nation of Waterridge
{'nation': 'Waterridge', 'national fealty': 1}


In [4]:
world.nations

{0: 'Waterridge',
 1: 'Earthwish',
 2: 'Rattown',
 3: 'Heartcraft',
 4: 'Stormloch',
 5: 'Oakmoral',
 6: 'Lionbrook',
 7: 'Lilycombe'}

In [6]:
nations.Nation("Waterridge",world)

TypeError: getCapitol_str() missing 1 required positional argument: 'towns'

In [ ]:
all_nations = [nations.Nation(n,world) for n in world.nations.values()]

In [ ]:
[t.type for t in world.towns] 

In [ ]:
My hope si that this will break

In [ ]:
np.random.choice(n.towns)

In [ ]:
n.ruler.temperment

In [ ]:
nations = []
for x in landscape['nations'].values():
    n = nation(x)
    n.addTowns(towns)
    n.addDiplomacy(landscape['nations'])
    n.appointRuler()
    nations.append(n)

In [ ]:
nations

In [ ]:
nations[0].name,nations[0].diplomacy

In [ ]:
def getNation(a):
    return [n for n in nations if n.name ==a][0]

def alterFavor(a,o,s):
    """
    a = the target nation (obj). a will not change. O's favor of A will change.
    o = the nation(s) who's favor is change. O's favor of a will change
    (a,[o]) each nation in o's favor of a is changed by s
    (a,o) o's favor of a is changed by s
    ([a],o) o's favor of each nation in a is changed by s
    s = amount of change (int)
    """
    if type(o) == list:
        for i in o:
            i.diplomacy[a.name]['favor'] += s
    elif type(a) == list:
         for i in a:
            o.diplomacy[i.name]['favor'] += s       
    else:
        o.diplomacy[a.name]['favor'] += s
    
getNation('Clearward').diplomacy

In [ ]:
alterFavor(getNation('Godtown'),[getNation('Clearward'),getNation('Clearcombe')],-1)    
getNation('Clearward').diplomacy

In [ ]:
nations[1].getCapitol_str()

In [ ]:
events = pd.DataFrame(columns=['n_subjects',
                              'n_objects',
                              'event',
                              'effect_var',
                              'effect'])
events.loc['raiders'] = [1,3,'raiders from {a}, plundered {o}','favor',-1]
events.loc['marrage'] = [1,1,'a princess from {o} was married to {a}','favor',3]
events.loc['temple'] = [1,0,'a great temple was founded in the nation of {a}','buildings','temple']
events

In [ ]:
landscape['eon'] = 10
landscape['chaos'] = .6

In [ ]:
def wheel_of_fate(events,nations,verbose=True):
    """
    a = the subject: the agent, or place where the event happened
    o = the object: the reciever of the event
    """
    choice = np.random.choice(events.index)
    event = events.loc[choice]
    a = np.random.choice(nations,event.n_subjects,replace=False)
    o = np.random.choice(nations,event.n_objects,replace=False)
    if all([len(a)>0,len(o)>0]):
        for ai in a:
            alterFavor(ai,[oi for oi in list(o) if oi!=ai],event.effect)         
    if verbose:
        text = (event.event.replace('{o}',str(o)).replace('{a}',str(a)))    
        print(text)
    
    
#now to run through the eons and let fate happen
for e in range(landscape['eon']):
    if np.random.random_sample()<landscape['chaos']:
        wheel_of_fate(events,nations)
    else:
        print(f'{e}: \'twas a quiet time')

In [ ]:
getNation('Godtown').diplomacy